In [186]:
import pandas as pd
import numpy as np

In [187]:
df_final = pd.read_csv('../raw_data/Excel_files/Margin_result_PE_analysis_PC2.csv')

In [188]:
print(df_final.head())

   Unnamed: 0   item_code  avg_conv_rate  last_price   last_date  \
0           0  1000024320       0.067821      119.99  2020-12-01   
1           1  1000005727       0.029961       99.99  2020-12-09   
2           2  1000007822       0.019591      279.99  2020-05-27   
3           3  1000114877       0.014925      179.99  2020-12-09   
4           5  1000008470       0.043734       99.99  2020-12-09   

   avg_unique_views      PE  PE_err      pc2  price_down  conv_rate_down  \
0         22.354839 -0.0854  0.0180   65.109  117.614039        0.067937   
1        176.119048 -0.0460  0.0073   41.450   97.034849        0.030002   
2        112.271605 -0.0246  0.0073  109.170  274.445827        0.019600   
3        193.820216 -0.0218  0.0049   77.859  176.425959        0.014932   
4         87.135135 -0.0643  0.0105   49.380   97.034849        0.043819   

   conv_rate_down_err  Delta_price  Margin_down  Margin_base  Margin_down_err  \
0            0.000024    35.722933    54.253161    54

In [189]:
df_final = df_final[['item_code','last_price','price_down','Margin_rate_to_base','PE','PE_err','pc2']]

In [190]:
df_final['Margin_rate_to_base']=df_final['Margin_rate_to_base']*100

In [191]:

df_final.head()

,item_code,last_price,price_down,Margin_rate_to_base,PE,PE_err,pc2
0,1000024320,119.99,117.614039,0.1708,-0.0854,0.0180,65.109
1,1000005727,99.99,97.034849,0.1380,-0.0460,0.0073,41.450
2,1000007822,279.99,274.445827,0.0492,-0.0246,0.0073,109.170
3,1000114877,179.99,176.425959,0.0436,-0.0218,0.0049,77.859
4,1000008470,99.99,97.034849,0.1929,-0.0643,0.0105,49.380


In [192]:
df_final=df_final.rename(columns={"last_price": "actual_price", "price_down": "optimal_price"})

In [193]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('read_tables').getOrCreate()


In [194]:
df_d_item = spark.read.parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*')


In [195]:
df_d_item.createOrReplaceTempView('d_item')

In [196]:
df_d_item.show()

+---------------+------------------+---------------------+-------------+----------------+----------+-------------+-----------+----------+-----------+--------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+------------------+-------------------+-------------------+---------+--------------------------------+---------------------------------------------+-----------------------+---------------------+------------------------+--------------------------+------------------------+--------------------+-----------+-------------------+------------------+-------------------+----------------------------+---------------+-----------------------+-----------------------+----------------------+------------------------------------+-------------------------------------------+-------------------------------------------------+--------------------------------------------------------+
|      item_skey|         item_code|item_parent_i

In [197]:
df_spark_final = spark.createDataFrame(df_final)


In [198]:
df_spark_final.createOrReplaceTempView('margin')

In [199]:
sp_df_pc2 = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/f_item_status_last/*.parquet',
    header=True)
sp_df_pc2.show()

+-----------+---------+----------------+-----------------------+--------------------------+--------------------------+--------------------------+--------------------------------+---------------------------------+---------------------------------------+
|  item_skey|shop_skey|item_status_skey|item_status_weekly_skey|item_pc1_costs_before_cogs|item_pc2_costs_before_cogs|group_pc2_margin_simulated|group_pc2_margin_ratio_simulated|group_pc2_margin_simulated_pocket|group_pc2_margin_ratio_simulated_pocket|
+-----------+---------+----------------+-----------------------+--------------------------+--------------------------+--------------------------+--------------------------------+---------------------------------+---------------------------------------+
| 4226129468|        1|               4|                      4|                    26.070|                    28.239|                      null|                            null|                             null|                             

In [200]:
sp_df_pc2.createOrReplaceTempView('f_item_last_status')

In [201]:
df_d_item_table_margin = spark.sql("""SELECT    margin.*,
                                                di.item_parent_item_code, di.item_main_category, di.item_inventory_flag, di.item_gtin_comparable,di.item_exclusivity,di.item_skey,
                                                fils.item_pc1_costs_before_cogs as pc1_costs
                                                from margin
                                                INNER JOIN d_item as di on di.item_code = margin.item_code
                                                INNER JOIN f_item_last_status as fils on fils.item_skey=di.item_skey and fils.shop_skey=1
                                                """)

df_d_item_table_margin.show()

+----------+------------+------------------+--------------------+-------+--------------------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+
| item_code|actual_price|     optimal_price| Margin_rate_to_base|     PE|              PE_err|               pc2|item_parent_item_code|  item_main_category|item_inventory_flag|item_gtin_comparable|item_exclusivity|      item_skey|pc1_costs|
+----------+------------+------------------+--------------------+-------+--------------------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+
|1000024320|      119.99|117.61403881007757| 0.17080000000001125|-0.0854|0.018000000000000002|            65.109|   000000008000002133|MATRATZEN&LATTENR...|               laso|          comparable| thirdPartyBrand| 31827848594342|   53.299|
|1000005727|       99.99| 97.0348488

In [202]:
df_d_item_table_margin.createOrReplaceTempView('item_margin')

In [203]:
df_standalone_item = spark.sql("""SELECT *
                from item_margin
                where item_parent_item_code like '%-P' """)

#127 items

In [204]:
df_pd_sa_item=df_standalone_item.toPandas()

In [205]:
shuffled = df_pd_sa_item.sample(frac=1)
result = np.array_split(shuffled, 3)

result[0].to_csv('../raw_data/Excel_files/Margin_result_standalone_test_1st.csv')
result[1].to_csv('../raw_data/Excel_files/Margin_result_standalone_test_2nd.csv')
result[2].to_csv('../raw_data/Excel_files/Margin_result_standalone_test_3rd.csv')

In [206]:
df_pd_sa_item['item_code'].head()


0    1000007822
1    1000182807
2    1000065698
3    1000054183
4    1000063634
Name: item_code, dtype: int64

In [207]:
item_sa_list = list(df_pd_sa_item['item_code'])

In [208]:
print(item_sa_list)

[1000007822, 1000182807, 1000065698, 1000054183, 1000063634, 1000092325, 1000111508, 1000053479, 1000123698, 1000051350, 1000118214, 1000008048, 1000123220, 1000050850, 1000056199, 1000008748, 1000119757, 1000110022, 1000118448, 1000110390, 1000145703, 1000015609, 1000007385, 1000048014, 1000054182, 1000086507, 1000117698, 1000039125, 1000065722, 1000022035, 1000114113, 1000164750, 1000123519, 1000008815, 1000183892, 1000015538, 1000098236, 1000181283, 1000007403, 1000112672, 1000008729, 1000015607, 1000007705, 1000034837, 1000119632, 1000008531, 1000065695, 1000150715, 1000123758, 1000129701, 1000120238, 1000015906, 1000008532, 1000008204, 1000050849, 1000008807, 1000008513, 1000117164, 1000190260, 1000189234, 1000039134, 1000008509, 1000123450, 1000092125, 1000065870, 1000069675, 1000039133, 1000007405, 1000008170, 1000008518, 1000015943, 1000118981, 1000192625, 1000007722, 1000118957, 1000048012, 1000014946, 1000033640, 1000127878, 1000008049, 1000048010, 1000008754, 1000054035, 100

In [209]:
pd_d_item_margin = df_d_item_table_margin.toPandas()




In [210]:
import psycopg2

def connect_redshift_db():
    cnx = psycopg2.connect(dbname= 'dwh',
                     host='redshift-prod.data.service.home24.net',
                     port= '5439',
                     user= 'gabriele_sabato',
                     password= 'zQZ8x6ZhRoTf7GS')
    return cnx
cnx_redshift = connect_redshift_db()
cursor_redshift = cnx_redshift.cursor()

In [211]:
def query_to_dataframe(_cursor, _query):
    _cursor.execute(_query)
    colnames = [desc[0] for desc in _cursor.description]
    print(colnames)
    return pd.DataFrame(_cursor.fetchall(), columns=colnames)

In [212]:
item_list=list(pd_d_item_margin['item_code'])
new_list = []
final_list = ""

for i in item_list:
    new_list.append(" '00000000" + str(i)+"' , " )

print(new_list)
final_list = final_list.join(new_list)
final_list=final_list[0:-3]

[" '000000001000024320' , ", " '000000001000005727' , ", " '000000001000007822' , ", " '000000001000114877' , ", " '000000001000008470' , ", " '000000001000022615' , ", " '000000001000182807' , ", " '000000001000008474' , ", " '000000001000008491' , ", " '000000001000174114' , ", " '000000001000061845' , ", " '000000001000065698' , ", " '000000001000022482' , ", " '000000001000054183' , ", " '000000001000173830' , ", " '000000001000016092' , ", " '000000001000063634' , ", " '000000001000092325' , ", " '000000001000111508' , ", " '000000001000138433' , ", " '000000001000012569' , ", " '000000001000016014' , ", " '000000001000053479' , ", " '000000001000123698' , ", " '000000001000016094' , ", " '000000001000016096' , ", " '000000001000051350' , ", " '000000001000118214' , ", " '000000001000118866' , ", " '000000001000005736' , ", " '000000001000008048' , ", " '000000001000123220' , ", " '000000001000016021' , ", " '000000001000050850' , ", " '000000001000056199' , ", " '0000000010000064

In [213]:
print(final_list)

 '000000001000024320' ,  '000000001000005727' ,  '000000001000007822' ,  '000000001000114877' ,  '000000001000008470' ,  '000000001000022615' ,  '000000001000182807' ,  '000000001000008474' ,  '000000001000008491' ,  '000000001000174114' ,  '000000001000061845' ,  '000000001000065698' ,  '000000001000022482' ,  '000000001000054183' ,  '000000001000173830' ,  '000000001000016092' ,  '000000001000063634' ,  '000000001000092325' ,  '000000001000111508' ,  '000000001000138433' ,  '000000001000012569' ,  '000000001000016014' ,  '000000001000053479' ,  '000000001000123698' ,  '000000001000016094' ,  '000000001000016096' ,  '000000001000051350' ,  '000000001000118214' ,  '000000001000118866' ,  '000000001000005736' ,  '000000001000008048' ,  '000000001000123220' ,  '000000001000016021' ,  '000000001000050850' ,  '000000001000056199' ,  '000000001000006422' ,  '000000001000008748' ,  '000000001000022411' ,  '000000001000024310' ,  '000000001000119757' ,  '000000001000119213' ,  '00000000100000

In [214]:
campaign_flag = """SELECT item_code,item_campaign_category, item_campaign_key, item_campaign_label
                    FROM dwh.public.item_catalog
                    WHERE item_code in ({FINAL_LIST})
                        and app_domain_id=1 """.format(FINAL_LIST=final_list)



In [215]:
df_campaign = query_to_dataframe(cursor_redshift, campaign_flag)



['item_code', 'item_campaign_category', 'item_campaign_key', 'item_campaign_label']


In [216]:
df_campaign.head()

,item_code,item_campaign_category,item_campaign_key,item_campaign_label
0,000000001000006076,|,wow12|crm-favdining,EXTRA -12 %|crm
1,000000001000008016,2019032601,3dmodels,3dmodels
2,000000001000008032,,,
3,000000001000008509,2020061801|,sidewow|test,EXTRA -12 %|Test Valerie
4,000000001000008729,,,


In [219]:
item_skey_list = list(pd_d_item_margin['item_skey'])
string_ints = [str(int) for int in item_skey_list]
str_of_ints = ",".join(string_ints)
print(str_of_ints)



31827848594342,71488185050378,224785681040185,25038272605281,66003517992621,74462021394914,100679798534570,154928563530175,73662191637457,169682504248479,2692801778218,135975807930039,138994088712555,46842188366232,120211027959520,20395642409368,65394400347680,127226683695048,20211508201120,128820719966683,187786938344533,227315835050470,234030476658978,108213988466089,141224110778088,151983822903626,198292212567208,216387891363420,229844485949139,134464046095856,138230763214441,129323200930102,139976088621728,112618974590311,274875034798182,2066942130973,146728563835247,112186534043649,82894169698233,260759150426570,220427820739992,187990528223809,69176282043018,170043872779977,99929938066990,237753328512171,38894203764772,187933000919936,52261297703691,43519991060918,258301027523634,155541362292937,207973439231826,231320324933583,155335976186971,208839644178623,178117027160469,122852719934137,40600078796222,131660168656219,12266273821396,46777765603668,243711583123936,334741852012,21

In [222]:
last_price_rule = """
SELECT item_skey, item_final_price_rule, period_date_skey--item_final_price_rule
  from dwh.star_analytical.f_item_attribute_history
 where shop_skey = 1
   and period_date_skey = (SELECT MAX(period_date_skey)
                             FROM dwh.star_analytical.f_item_attribute_history)
   and item_skey in ({ITEM_SKEY})
""".format(ITEM_SKEY=str_of_ints)

In [223]:
df_last_price_rule = query_to_dataframe(cursor_redshift, last_price_rule)

['item_skey', 'item_final_price_rule', 'period_date_skey']


In [224]:
sp_last_price_rule = spark.createDataFrame(df_last_price_rule)
sp_last_price_rule.createOrReplaceTempView('last_price_rule')
sp_campaign = spark.createDataFrame(df_campaign)
sp_campaign.createOrReplaceTempView('campaign')

In [231]:
final_df_margin = spark.sql("""SELECT im.*,
                    lpr.item_final_price_rule,
                    camp.item_campaign_key,
                    camp.item_campaign_label
                    from item_margin as im
                    INNER JOIN last_price_rule as lpr on lpr.item_skey = im.item_skey
                    INNER JOIN campaign as camp on CAST(camp.item_code as int) = im.item_code
                    """)

In [241]:
final_df_margin.show()

+----------+------------+------------------+--------------------+--------------------+------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+---------------------+--------------------+--------------------+
| item_code|actual_price|     optimal_price| Margin_rate_to_base|                  PE|PE_err|               pc2|item_parent_item_code|  item_main_category|item_inventory_flag|item_gtin_comparable|item_exclusivity|      item_skey|pc1_costs|item_final_price_rule|   item_campaign_key| item_campaign_label|
+----------+------------+------------------+--------------------+--------------------+------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+---------------------+--------------------+--------------------+
|1000016121|      307.99| 301.8913893917476| 0.04020000000001083|             -0.0201| 0

In [250]:
df_margin_panda= final_df_margin.toPandas()


In [251]:
df_wow = df_margin_panda[df_margin_panda['item_campaign_key'].str.contains('wow')]

In [252]:
print(len(df_wow['item_code']))

193


In [253]:
df_not_wow = df_margin_panda[~df_margin_panda['item_campaign_key'].str.contains('wow')]

In [254]:
print(len(df_not_wow))

199


In [255]:
df_not_wow_sa = df_not_wow[df_not_wow['item_parent_item_code'].str.contains('-P')]

In [256]:
print(len(df_not_wow_sa))

74


In [259]:
df_sa = df_margin_panda[df_margin_panda['item_parent_item_code'].str.contains('-P')]

In [260]:
print(len(df_sa))

127


In [261]:

df_margin_panda.to_csv('../raw_data/Excel_files/Margin_result_all_in_v0.csv')